# Intraday Data

- Reads in intraday stock data for 5 minute samples from csv files where each dataset contains different lengths of data. 
- Removes days with insufficient price information
- Outputs a dataframe containing intraday price data for 9 different stocks

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
tickers = ['AAPL', 'AMD', 'BAC', 'CSCO', 'GE', 'MSFT', 'MU', 'T', 'WFC']

In [3]:
# Reads in intraday price data for a given stock and cleans it to start from 2000
def get_intraday_data(name):
    file_src  = 'D:/Thesis/' + name + '-5m.csv'
    df = pd.read_csv(file_src, sep = ';', header = None, 
                     names = ['Date', 'Time', 'Open', 'High', 'Low', name, 'Volume'])
    df['Datetime'] = df['Date'] + ' ' + df['Time']
    df['Datetime'] = pd.to_datetime(df['Datetime'], format = '%d/%m/%Y %H:%M:%S')
    df = df[df.Datetime.dt.year >= 2000]
    return df

In [4]:
AAPL = get_intraday_data(tickers[0])
AMD = get_intraday_data(tickers[1])
BAC = get_intraday_data(tickers[2])
CSCO = get_intraday_data(tickers[3])
GE = get_intraday_data(tickers[4])
MSFT = get_intraday_data(tickers[5])
MU = get_intraday_data(tickers[6])
T = get_intraday_data(tickers[7])
WFC = get_intraday_data(tickers[8])

In [5]:
# See which days there is price data on for a given dataset
def get_trading_days(df):
    days = df.Date.value_counts().index
    return set(days)

In [6]:
# See which days do not have full price data for a given dataset
def get_partial_trading_days(df):
    days = df.Date.value_counts().index[(df.Date.value_counts() < 78) == True]
    return set(days)

In [7]:
data = [AAPL, AMD, BAC, CSCO, GE, MSFT, MU, T, WFC]
partial_days = set()

In [8]:
# Get set of all days where for some stock there is incomplete price data
for df in data:
    days = get_partial_trading_days(df)
    partial_days = partial_days.union(days)

In [9]:
# Get set of all common trading days regardless of amount price data
trading_days = set(WFC.Date.unique())

for df in data:
    days = get_trading_days(df)
    trading_days = trading_days.intersection(days)

In [10]:
# Calculate set of days where full price data is available
full_days = trading_days - partial_days

In [11]:
# Filter datasets to only contain dates with full price information
AAPL = AAPL[AAPL.Date.isin(list(full_days))]
AMD  = AMD[AMD.Date.isin(list(full_days))]
BAC  = BAC[BAC.Date.isin(list(full_days))]
CSCO = CSCO[CSCO.Date.isin(list(full_days))]
GE   = GE[GE.Date.isin(list(full_days))]
MSFT = MSFT[MSFT.Date.isin(list(full_days))]
MU   = MU[MU.Date.isin(list(full_days))]
T    = T[T.Date.isin(list(full_days))]
WFC  = WFC[WFC.Date.isin(list(full_days))]

In [12]:
# Duplicates however remain and need to be removed
CSCO_duplicates = CSCO[CSCO.duplicated()].index.tolist()
GE_duplicates = GE[GE.duplicated()].index.tolist()
MU_duplicates = MU[MU.duplicated()].index.tolist()

CSCO.drop(CSCO_duplicates, inplace = True)
GE.drop(GE_duplicates, inplace = True)
MU.drop(MU_duplicates, inplace = True)

C:\Users\manni\anaconda3\envs\Trading\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
# Some days however have 79 observations for a day where data is also given for 16:00 and needs to be removed 
# since not all stocks have this data 
def get_extra_data_idx(df):
    idx = list()
    extra_days = (df.Date.value_counts().index[(df.Date.value_counts() == 79) == True]).tolist()
    
    for day in extra_days:
        idx.append(df[df.Date == day].index[-1])
    
    return idx

In [14]:
# Calculate index for each 79th price observation
AAPL_extra_idx = get_extra_data_idx(AAPL)
AMD_extra_idx = get_extra_data_idx(AMD)
BAC_extra_idx = get_extra_data_idx(BAC)
CSCO_extra_idx = get_extra_data_idx(CSCO)
GE_extra_idx = get_extra_data_idx(GE)
MSFT_extra_idx = get_extra_data_idx(MSFT)
MU_extra_idx = get_extra_data_idx(MU)
T_extra_idx = get_extra_data_idx(T)
WFC_extra_idx = get_extra_data_idx(WFC)

In [15]:
# AAPL has one day where 91 observations are made with data from 8:30 to 9:25 is calculated and 16:00 is reported
aapl_91_idx = AAPL[AAPL.Date == '07/07/2017'][:12].index.tolist()
aapl_91_idx = aapl_91_idx + [AAPL[AAPL.Date == '07/07/2017'].index[-1]]

for idx in aapl_91_idx:
    AAPL_extra_idx.append(idx)

In [16]:
# Drop all extra price data
AAPL.drop(AAPL_extra_idx, inplace = True)
AMD.drop(AMD_extra_idx, inplace = True)
BAC.drop(BAC_extra_idx, inplace = True)
CSCO.drop(CSCO_extra_idx, inplace = True)
GE.drop(GE_extra_idx, inplace = True)
MSFT.drop(MSFT_extra_idx, inplace = True)
MU.drop(MU_extra_idx, inplace = True)
T.drop(T_extra_idx, inplace = True)
WFC.drop(WFC_extra_idx, inplace = True)

C:\Users\manni\anaconda3\envs\Trading\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
AAPL.reset_index(drop = True, inplace = True)
AMD.reset_index(drop = True, inplace = True)
BAC.reset_index(drop = True, inplace = True)
CSCO.reset_index(drop = True, inplace = True)
GE.reset_index(drop = True, inplace = True)
MSFT.reset_index(drop = True, inplace = True)
MU.reset_index(drop = True, inplace = True)
T.reset_index(drop = True, inplace = True)
WFC.reset_index(drop = True, inplace = True)

In [23]:
# Join all dataset together
data = AAPL[['Date', 'AAPL']]
data = data.join([AMD['AMD'], BAC['BAC'], CSCO['CSCO'], GE['GE'], MSFT['MSFT'], MU['MU'], T['T'], WFC['WFC']])
data = data.join(AMD.Datetime)
data.drop('Date', axis = 1, inplace = True)

In [26]:
data.to_csv('D:/Thesis/IntradayReturns.csv')